In [1]:
import pandas as pd
import numpy as np

In [2]:
ratings = pd.read_csv('train_dt/ratings_small.csv')

movies_metadata = pd.read_csv('train_dt/movies_metadata.csv')

credits = pd.read_csv('train_dt/credits.csv')

keywords = pd.read_csv('train_dt/keywords.csv')

links = pd.read_csv('train_dt/links.csv')

C:\Users\chado\AppData\Local\Temp\ipykernel_12080\316963216.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('train_dt/movies_metadata.csv')


In [3]:
# ratings preprocessing

#추후 merge를 위해 movieId를 id로 통일
ratings.rename(columns={'movieId':'id'}, inplace=True)
#추후 merge를 위해 index id의 값들을 숫자형태로 변경
ratings.loc['id'] = ratings['id'].astype('str')
ratings['id'] = pd.to_numeric(ratings['id'])

In [4]:
# movies_metadata preprocessing

#나중에 최종결과 보여줄 때 movies_metadata_origin을 사용할 예정
movies_metadata_origin = movies_metadata

#계산에 불필요한 ft drop
movies_metadata.drop(['homepage','overview', 'tagline', 'poster_path'], axis = 1, inplace = True)

#이 두 ft는 kaggle에는 안뜨는 데 다운받으면 있음
#의미를 몰라서 일단 drop
movies_metadata.drop(['vote_average','vote_count'], axis = 1, inplace = True)

#movies_metadata_origin[movies_metadata_origin['production_companies'] == "[]"] -> 결과 : 11559 
#null값이 11559개(전체 movie dataset : 45031개)라 총 1/4정도를 차지해서 없애는 것이 낫다고 판단
movies_metadata.drop(['production_companies'], axis = 1, inplace = True)

#belongs_to_collection null 값을 no collection 값으로 변경
movies_metadata['belongs_to_collection'].replace(np.nan,'no collection', inplace = True)

#null값 없애기
movies_metadata.dropna(inplace = True)

#'genres', 'proudction_countries'가 null인 경우 drop
movies_metadata = movies_metadata.loc[movies_metadata_origin['genres'] != "[]"]
movies_metadata = movies_metadata.loc[movies_metadata_origin['production_countries'] != "[]"]

movies_metadata['budget'] = pd.to_numeric(movies_metadata['budget'])

#movies_metadata['budget'] 0인 것은 나머지의 평균으로 넣는다.
budget_mean = movies_metadata.loc[movies_metadata['budget'] != 0]['budget'].mean()
movies_metadata['budget'].replace(0, budget_mean, inplace = True)

#'id'형식이 날짜 형식인 경우 -> 잘못된 경우
#ex) ValueError: Unable to parse string "2012-09-29" at position 29502
movies_metadata['isIdRight'] = movies_metadata['id'].str.contains('|'.join('-'))
movies_metadata = movies_metadata[movies_metadata['isIdRight'] == False]
movies_metadata.drop(['isIdRight'], axis = 1, inplace = True)

#추후 merge를 위해 index id의 값들을 숫자형태로 변경
movies_metadata.loc['id'] = movies_metadata['id'].astype('str')
movies_metadata['id'] = pd.to_numeric(movies_metadata['id'])

In [5]:
# credits preprocessing

# credits의 [] - null 값 제거
credits = credits.loc[credits['cast'] != "[]"]
credits = credits.loc[credits['crew'] != "[]"]

#추후 merge를 위해 index id의 값들을 숫자형태로 변경
credits['id'] = pd.to_numeric(credits['id'])

# keywords preprocessing
keywords = keywords.loc[keywords['keywords'] != "[]"]

# link preprocessing
links = links.dropna() #45843 -> 45624


In [8]:
# merging for making complete training dataset
all_movie_df = pd.merge(movies_metadata, credits)
all_movie_df.head(5)#null값 없음

,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,original_title,popularity,production_countries,release_date,revenue,runtime,spoken_languages,status,title,video,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",3.000000e+07,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862.0,tt0114709,en,Toy Story,21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Toy Story,False,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,no collection,6.500000e+07,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844.0,tt0113497,en,Jumanji,17.015539,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Jumanji,False,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",2.232716e+07,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602.0,tt0113228,en,Grumpier Old Men,11.7129,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Grumpier Old Men,False,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,False,no collection,1.600000e+07,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357.0,tt0114885,en,Waiting to Exhale,3.859495,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Waiting to Exhale,False,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",2.232716e+07,"[{'id': 35, 'name': 'Comedy'}]",11862.0,tt0113041,en,Father of the Bride Part II,8.387519,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Father of the Bride Part II,False,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [10]:
all_movie_df = pd.merge(all_movie_df, ratings)
all_movie_df.head(5)

,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,original_title,popularity,production_countries,...,runtime,spoken_languages,status,title,video,cast,crew,userId,rating,timestamp
0,False,no collection,60000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",949.0,tt0113277,en,Heat,17.924927,"[{'iso_3166_1': 'US', 'name': 'United States o...",...,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Heat,False,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",23.0,3.5,1.148721e+09
1,False,no collection,60000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",949.0,tt0113277,en,Heat,17.924927,"[{'iso_3166_1': 'US', 'name': 'United States o...",...,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Heat,False,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",102.0,4.0,9.565989e+08
2,False,no collection,60000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",949.0,tt0113277,en,Heat,17.924927,"[{'iso_3166_1': 'US', 'name': 'United States o...",...,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Heat,False,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",232.0,2.0,9.550927e+08
3,False,no collection,60000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",949.0,tt0113277,en,Heat,17.924927,"[{'iso_3166_1': 'US', 'name': 'United States o...",...,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Heat,False,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",242.0,5.0,9.566888e+08
4,False,no collection,60000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",949.0,tt0113277,en,Heat,17.924927,"[{'iso_3166_1': 'US', 'name': 'United States o...",...,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Heat,False,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",263.0,3.0,1.117847e+09


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# get top 15000 movies based on popularity
#all_movie_df = movies_metadata.sort_values('popularity', ascending=False).reset_index(drop=True)
#all_movie_df = all_movie_df.iloc[:15000].copy()

# drop duplicates based on title
#all_movie_df.drop_duplicates(subset=['title'], inplace=True)


In [ ]:
# feature => 'overview'

# replace NaN values with empty string
all_movie_df['overview'] = all_movie_df['overview'].fillna('')

# cleaning the text
# all_movie_df['overview'] = all_movie_df['overview'].progress_apply(text_process)

# convert overview column data to matrix to be able to compute similarity after removing the stop words
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=0.0001)
tfidf_matrix = tfidf.fit_transform(all_movie_df['overview'])

print(tfidf_matrix.shape)

# compute similarity matrix
# options available - euclidean distance, the Pearson Coeffiecient, cosine similarity score

# compute cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# create reverse title indices series
indices = pd.Series(all_movie_df.index, index=all_movie_df['title'])

In [ ]:
# feature => 'tagline'

# replace NaN values with empty string
all_movie_df['tagline'] = all_movie_df['tagline'].fillna('')

# cleaning the text
# all_movie_df['tagline'] = all_movie_df['tagline'].progress_apply(text_process)

# convert overview column data to matrix to be able to compute similarity after removing the stop words
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=0.0001)
tfidf_matrix = tfidf.fit_transform(all_movie_df['tagline'])

print(tfidf_matrix.shape)

# compute similarity matrix
# options available - euclidean distance, the Pearson Coeffiecient, cosine similarity score

# compute cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# create reverse title indices series
indices = pd.Series(all_movie_df.index, index=all_movie_df['title'])

In [ ]:
# feature => 'genres', 'production_companies', 'production_countries' ...

In [ ]:
def get_recommendation(title):
    # get index of given title
    idx = indices[title]
    
    # get similarity scores along with indices
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # sort the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # get top 10 similarity scores
    sim_scores = sim_scores[1:11]
    
    # get movie indices
    movie_indices = [i[0] for i in sim_scores]
    movie_scores = [i[1] for i in sim_scores]
    
    # get movie titles
    movies = all_movie_df.iloc[movie_indices]['title']
    
    return pd.DataFrame({'Recommended Movie':movies, 'Similarity Score':movie_scores})

In [ ]:
get_recommendation('Toy Story')